In [48]:
%reload_ext autoreload
%autoreload 2
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv
from gridworld_env_2d_state import GridworldEnv2DState
import torch
import random
import numpy as np
import math
from EXITrl.table_base import TableBase
from EXITrl.approx_v_base import ApproxVBase
from EXITrl.approx_policy_base import ApproxPolicyBase
from EXITrl.table_base import TableBase
from EXITrl.base import Base
import gym

In [54]:
class ActorCriticQ(ApproxVBase, ApproxPolicyBase):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        hidden = 8
        features = torch.nn.Linear(self.num_state, hidden)
        model_q = torch.nn.Sequential(
            features,
            torch.nn.ReLU(),
            torch.nn.Linear(hidden, hidden),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden, self.num_action),
        ).to(self.device)
        model_policy = torch.nn.Sequential(
            features,
            torch.nn.ReLU(),
            torch.nn.Linear(hidden, hidden),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden, self.num_action),
        ).to(self.device)
        ApproxVBase.initialize(self, learning_rate_name="beta", model=model_q)
        ApproxPolicyBase.initialize(self, learning_rate_name="alpha", model=model_policy)
    
    def _loop(self, episode) -> int:
        done = False
        total_reward, reward = 0, 0
        state = self.env.reset()
        action, log_prob = self.policy(state)
        for i in range(1000):
            _state, reward, done, _ = self.env.step(action)
            _action, _log_prob = self.policy(_state)
            if done: 
                td_target = reward
            else: 
                td_target = reward + self.gamma * self.get_q(_state)[_action]
    
            # policy improvement
            # 1. Sutton's book
#             loss = (-log_prob) * self.gamma**i
            # 2. David's slide
            loss = (-log_prob) * self.get_q(state)[action]
            self.update_policy(loss)
            
            # value evaluation
#             td_error = td_target - self.get_q(state, action)
#             self.Q[state, action] += self.alpha * td_error
            current_q = self.get_q(state)[action]
            self.update_q(td_target, current_q)

            total_reward += reward
            state = _state
            action = _action
            log_prob = _log_prob
            if done: return total_reward

try: env.close()
except: pass
env = gym.make('CartPole-v1')
# env = GridworldEnv2DState()
# env = GridworldEnv()
s = ActorCriticQ(env, 
               num_episodes=500,
               policy="softmax_policy",
               alpha=0.007, 
               beta=0.1,
               gamma=.99)
s.train(True)
# s.convert_Q_to_V()


/Users/epinyoanun/miniconda3/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
episode: 0 reward: 11.0
episode: 1 reward: 9.0
episode: 2 reward: 21.0
episode: 3 reward: 18.0
episode: 4 reward: 9.0
episode: 5 reward: 13.0
episode: 6 reward: 9.0
episode: 7 reward: 10.0
episode: 8 reward: 9.0
episode: 9 reward: 9.0
episode: 10 reward: 9.0
episode: 11 reward: 9.0
episode: 12 reward: 8.0
episode: 13 reward: 9.0
episode: 14 reward: 10.0
episode: 15 reward: 9.0
episode: 16 reward: 10.0
episode: 17 reward: 9.0
episode: 18 reward: 9.0
episode: 19 reward: 8.0
episode: 20 reward: 8.0
episode: 21 reward: 9.0
episode: 22 reward: 10.0
episode: 23 reward: 10.0
episode: 24 reward: 9.0
episode: 25 reward: 8.0
episode: 26 reward: 9.0
episode: 27 reward: 9.0
episode: 28 reward: 10.0
episode: 29 reward: 9.0
episode: 30 reward: 10.0
episode: 31 reward: 10.0
episode: 32 reward: 9.0
episode: 33 reward: 9.0
episode: 34 reward: 10.0
episode: 35 reward: 10.0
episode: 36 reward: 9.0
episode: 

episode: 323 reward: 9.0
episode: 324 reward: 10.0
episode: 325 reward: 9.0
episode: 326 reward: 8.0
episode: 327 reward: 9.0
episode: 328 reward: 9.0
episode: 329 reward: 9.0
episode: 330 reward: 10.0
episode: 331 reward: 10.0
episode: 332 reward: 11.0
episode: 333 reward: 9.0
episode: 334 reward: 10.0
episode: 335 reward: 9.0
episode: 336 reward: 9.0
episode: 337 reward: 10.0
episode: 338 reward: 9.0
episode: 339 reward: 9.0
episode: 340 reward: 10.0
episode: 341 reward: 8.0
episode: 342 reward: 9.0
episode: 343 reward: 8.0
episode: 344 reward: 9.0
episode: 345 reward: 10.0
episode: 346 reward: 9.0
episode: 347 reward: 10.0
episode: 348 reward: 10.0
episode: 349 reward: 10.0
episode: 350 reward: 10.0
episode: 351 reward: 9.0
episode: 352 reward: 8.0
episode: 353 reward: 10.0
episode: 354 reward: 10.0
episode: 355 reward: 10.0
episode: 356 reward: 9.0
episode: 357 reward: 10.0
episode: 358 reward: 10.0
episode: 359 reward: 9.0
episode: 360 reward: 9.0
episode: 361 reward: 9.0
episode: